Step 1:
    Create df of urls w/how many pages are in the topic.
Step 2:
    Create system for filling out those two things in new df.

In [11]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import requests
from time import sleep
from bs4 import BeautifulSoup
import json
from time import time
from datetime import datetime

In [5]:
def get_html(url):
    return BeautifulSoup(requests.get(url).content)

In [6]:
def urls_with_numbers(forum_page_url, forum_page_num = False):
    forum_soup     = get_html(forum_page_url)     # Input URL get Soup
    
    # List of urls of topics in forum page
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] 
        for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topics         = []
    count          = 0
    
    # for i in all the forum topic infos
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):
        # Turning each individual info bit into a dict
        posts_num   = json.loads(forum_topic.attrs['data-forum-topic']
            )['lastPosition'] 
        
        # Num of posts in Topic divided by num allowed per page +1 for 1st
        topic_pages = posts_num//20 + 1 
        
        if posts_num%20 != 0 and posts_num > 20:# If there's a remainder
            topic_pages += 1                    # Add remainder page
        topics.append({
            'url'      : topic_url_list[count],
            'pages'    : topic_pages,
            'Forum_num': forum_page_num
            })
        count += 1
    return topics # Returns list of dicts


In [12]:
dict_list = []
b_url = 'https://us.battle.net/forums/en/overwatch/22813879/?page='
for attempt in range(5):
    try:
        for forum_num in range(9999):
            current_list = urls_with_numbers(b_url + str(forum_num),
                forum_num)
            
            dict_list += current_list
            if forum_num % 100 == 0:
                print(forum_num, 'Added', 
                    datetime.fromtimestamp(time()
                    ).strftime('%H:%M:%S'))
    except:
        print("---------")
        print('Error', attempt)
        print("---------")
        sleep(100)
    pd.DataFrame(dict_list).to_csv('./data/urls' + str(attempt) + '.csv')

0 Added 18:54:55
100 Added 18:57:34
200 Added 19:00:12
300 Added 19:02:49
400 Added 19:05:30
500 Added 19:08:10
600 Added 19:10:51
700 Added 19:13:32
800 Added 19:16:15
900 Added 19:18:58
1000 Added 19:21:42
1100 Added 19:24:26
1200 Added 19:27:10
1300 Added 19:30:00
1400 Added 19:32:46
1500 Added 19:35:34
1600 Added 19:38:29
1700 Added 19:41:14
1800 Added 19:44:07
1900 Added 19:46:56
2000 Added 19:49:46
2100 Added 19:52:35
2200 Added 19:55:28
2300 Added 19:58:22
2400 Added 20:01:15
2500 Added 20:04:09
2600 Added 20:07:07
2700 Added 20:10:00
2800 Added 20:12:54
2900 Added 20:15:51
3000 Added 20:18:46
3100 Added 20:21:43
3200 Added 20:24:39
3300 Added 20:27:34
3400 Added 20:30:36
3500 Added 20:33:36
3600 Added 20:36:37
3700 Added 20:53:22
3800 Added 21:14:45
3900 Added 21:35:40
4000 Added 21:56:35
4100 Added 22:17:30
4200 Added 22:38:30
4300 Added 22:59:54
4400 Added 23:20:10
4500 Added 23:40:43
4600 Added 00:01:34
4700 Added 00:22:32
4800 Added 00:43:30
4900 Added 01:04:09
5000 Added 0

KeyboardInterrupt: 

In [10]:
dict_list = []
b_url = 'https://us.battle.net/forums/en/overwatch/22813879/?page='
for attempt in range(5):
    #try:
    for forum_num in range(9999):
        current_list = urls_with_numbers(b_url + str(forum_num),
            forum_num)

        dict_list += current_list
            #if forum_num % 100 == 0:
        print(forum_num, 'Added', 
            datetime.fromtimestamp(time()
            ).strftime('%H:%M:%S'))
    #except:
    #    print("---------")
    #    print('Error', attempt)
    #    print("---------")
    #    sleep(100)
    pd.DataFrame(dict_list).to_csv('./data/urls__.csv')#' + str(attempt) + '.csv')

AttributeError: module 'datetime' has no attribute 'fromtimestamp'

800 at about 7:31 ish
1300 at about 7:45 ish
1400 at about 7:48 ish 

In [35]:
print(len(test_list))
test_df = pd.DataFrame(test_list)

546


In [39]:
test_df.to_csv('./data/urls.csv')

In [5]:
def save_posts(list_of_dicts, saver, forum_page):
    return pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'stqatuses',
                            'topic_url']).to_csv(saver + str(forum_page), index=False)

In [6]:
words = []
dates = []
TopicPosts = []
title = []
forum_page = []
prof_link = []
auth_posts = []
topic_keys  = []
def turn_to_dict(post, words = words, 
                 dates = dates, TopicPosts = TopicPosts, 
                 title = title, forum_page = forum_page, 
                 prof_link = prof_link, auth_posts = auth_posts, topic_keys = topic_keys):
    post_dict = {}
    try:
        post_dict = {                               # Creation & Statement of dicts
            'text'      : words[post].contents,                     # Text of the post
            'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
            'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
            'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
            'title'     : title,                    # Title of Topic
            'forum_page': forum_page,               # Page in the forum
            'topic_url' : topic_keys[post][0]}
    except:
        pass
    
    try:
        post_dict['statuses']   = TopicPosts[post].attrs['data-topic']
    except:
        pass
    
    try:
        post_dict['prof_link']  = prof_link[post].attrs['href']
    except:
        pass
    
    try:
        post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
    except:
        pass
    
    return post_dict

In [7]:
edt = ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']

In [10]:
# THIS is for turning into the Requests library
list_of_dicts = []                                              # Instantiate empty list of DICKS to FILL UP
saver = './data/full_scrapes/Overwatch_Test_'                   # The path & starting name for saving junk

for forum_page in range(1,101):                                 # Will go through all the forum pages specified
    topic_keys = urls_with_numbers('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
    if forum_page % 100 == 0:
        save_posts(list_of_dicts, saver, forum_page) 
    for topic_stats in topic_keys:                              # For every title in the list
        for page in range(topic_stats[1]):                      # for page # in range of # of pages in the topic
            page += 1                                           # Add 1 to compensate for starting at 0
            if page > 1:                                        # Basically making sure it's not the first page
                URL = topic_stats[0] + '?page=' + str(page)     # This is basicslly for getting to all the pages
            else:
                URL = topic_stats[0]                            # This will be the first page
            
            # This chunk defines the lists of things that we want per row
            page_soup  = get_html(URL)                          # Soup of the page
            title      = page_soup.find(attrs={'class':'Topic-title'}).contents
            dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents !=edt]
            words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
            TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
            auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
            prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
            print("forum page:", forum_page, "Posts in Topic:", topic_stats[1],"Page:",page,"URL:",URL )
            for post in range(len(words)):                           # For each post that's present
                list_of_dicts.append(turn_to_dict(post))        # Add the dict of post to list of posts
        print("forum page:", forum_page,"Topic Page:", page, "Last URL:", URL) # Sanity check
df = save_posts(list_of_dicts, saver, forum_page)               # Should both save the data & create a df to see

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [10]:
!pip install lxml

In [ ]:
#!conda install beautifulsoup4
!conda install lxml
#!conda install -c conda-forge scrapy


Solving environment: \ 

In [14]:
# Tester to see if this will continue to run within different situations
for i in range(60):
    print(datetime.fromtimestamp(time()).strftime('%H:%M:%S'))
    sleep(30)

NameError: name 'y' is not defined

In [13]:
pd.DataFrame.from_csv('./data/urls0.csv')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,Forum_num,pages,url
0,0,1,https://us.battle.net/forums/en/overwatch/topi...
1,0,1,https://us.battle.net/forums/en/overwatch/topi...
2,0,238,https://us.battle.net/forums/en/overwatch/topi...
3,0,6,https://us.battle.net/forums/en/overwatch/topi...
4,0,10,https://us.battle.net/forums/en/overwatch/topi...
5,0,1,https://us.battle.net/forums/en/overwatch/topi...
6,0,1,https://us.battle.net/forums/en/overwatch/topi...
7,0,1,https://us.battle.net/forums/en/overwatch/topi...
8,0,1,https://us.battle.net/forums/en/overwatch/topi...
9,0,3,https://us.battle.net/forums/en/overwatch/topi...


In [24]:
def urls_with_numbers(forum_page_url, forum_page_num = False):
#current_forum_page = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
    forum_soup     = get_html(forum_page_url)                                       # Input URL get Soup
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] 
        for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topic_tuples   = []                                                           # Above is list of urls of topics in forum page
    count          = 0                                                                   # Instantiating empty list and starting count
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):      # for i in all th3e forum topic infos
        posts_num = json.loads(forum_topic.attrs['data-forum-topic']
            )['lastPosition'] # Turning each individual info bit into a dict
        topic_pages = posts_num//20 + 1                     # Number of posts in the Topic divided by num allowed per page +1 for 1st
        if posts_num%20 != 0 and posts_num > 20:            # If there's a remainder page
            topic_pages += 1                                # Add remainder page
        if forum_page_num == False:
            topic_tuples.append((topic_url_list[count],topic_pages)) # Add the url to the page amount in tuples
        else:
            topic_tuples.append((forum_page_num,topic_url_list[count],topic_pages))
        count += 1                                          # Keep track of the count
    return topic_tuples                                     # Returns list of tuples
